# Rating of a Plain Fin-Tube Counterflow Air Condenser
---

**Importing libraries and notebook preparation**

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CoolPropWarning

warnings.filterwarnings('ignore', category=CoolPropWarning)

Q_ = Quantity

from hvac.heat_exchanger.recuperator.fintube.continuous_fin import PlainFinTubeCounterFlowAirCondenser as Condenser

## Selection of the Refrigerant

Define the type of refrigerant:

In [2]:
R134a = Fluid('R134a')

## Definition of the Condenser

Define the type of condenser: a plain fin-tube, counterflow heat exchanger.

Create an instance of the condenser model, specifying the dimensions of the frontal area, the number of rows, the geometrical characteristics of the heat transfer surfaces, and the thermal conductivity of the fins. The dimensions of the frontal area were determined in our preliminary calculations (see *ex01_prelim.ipynb*).

In [3]:
condenser = Condenser(
    W_fro=Q_(1.003, 'm'),
    H_fro=Q_(0.334, 'm'),
    N_rows=5,
    S_trv=Q_(25.4, 'mm'),         # vertical distance between tubes
    S_lon=Q_(22.0, 'mm'),         # horizontal distance between tubes
    D_int=Q_(8.422, 'mm'),        # inner tube diameter
    D_ext=Q_(10.2, 'mm'),         # outer tube diameter
    t_fin=Q_(0.3302, 'mm'),       # fin thickness
    N_fin=1 / Q_(3.175, 'mm'),    # fin density
    k_fin=Q_(237, 'W / (m * K)')  # conductivity of fin material
)

## Rating the Condenser

On this object, we set the known operating conditions:
- the mass flow rate of air through the condenser
- the mass flow rate of refrigerant, which was determined when rating the evaporator
- the state of air that enters the condenser
- the state of refrigerant entering the condenser

The result of the condenser rating will contain:
- the state of refrigerant leaving the condenser
- the state of air leaving the condenser
- the heat rejection rate (the heat transfer rate from the refrigerant to the air stream)
- the degree of subcooling
- the flow length of air through the desuperheating, condensing, and subcooling part of the condenser

When we rated the evaporator (see notebook *ex02_air_evaporator.ipynb*) to get at the desired leaving air temperature (13 °C), we've noticed that the mass flow rate of refrigerant needed to superheat the refrigerant (10 K) was greater than the mass flow rate we had determined in our preliminary calculations (see notebook *ex01_prelim_calc.ipynb*): 168.745 kg/h instead of 138.854 kg/h. This was because the evaporator extracts more moisture from the air than we had assumed in our preliminary calculations, where we arbitrary assumed that the air leaving the evaporator would be nearly saturated (RH 90 %). To rate the condenser, we need the state of the refrigerant at the condenser's entrance (which we assume to be equal to the state of the refrigerant discharged by the compressor). Using the compressor selection program again, we can repeat the compressor selection procedure with the heat absorption rate that followed from the evaporator rating (keeping the evaporation temperature, the condensing temperature, and the degree of superheating and subcooling the same as before). This leads us now to a variable speed compressor DANFOSS VTZ054-G running at 4343 rpm and the estimated discharge temperature of the refrigerant is about 81.7 °C. We keep the condensing pressure that we determined in our preliminary calculations.  

We recalculate the heat rejection rate in the condenser:

In [4]:
# Refrigerant state at condenser inlet:
rfg_in = R134a(T=Q_(81.7, 'degC'), P=Q_(13.179, 'bar'))

# Selected condensing temperature:
T_cnd = Q_(50, 'degC')

rfg_sat = R134a(T=T_cnd, x=Q_(0, 'frac'))
P_cnd = rfg_sat.P

# Selected degree of subcooling:
dT_sc = Q_(5, 'K')

# Refrigerant state at condenser outlet:
T_rfg_out = rfg_sat.T - dT_sc 
rfg_out = R134a(T=T_rfg_out, P=P_cnd)

# Selected mass flow rate of refrigerant 
# (which follows from the evaporator rating)
rfg_m_dot = Q_(168.745, 'kg / hr')

# Heat rejection rate of refrigerant in the condenser:
cnd_Q_dot = rfg_m_dot * (rfg_in.h - rfg_out.h)
print(cnd_Q_dot.to('kW'))

9.174145837586556 kilowatt


And we recalculate the mass flow rate of air through the condenser:

In [5]:
# Air state at condenser inlet:
air_in = HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct'))

# Selected air temperature rise across condenser
# (this choice depends also on the selected condensing temperature):
air_dT = Q_(10, 'K')  

# Air state at condenser outlet:
air_out = HumidAir(Tdb=air_in.Tdb + air_dT, W=air_in.W)

# Mass flow rate of air through the condenser:
air_m_dot = cnd_Q_dot / (air_out.h - air_in.h)
print(air_m_dot.to('kg/hr'))

3216.3149327018023 kilogram / hour


Now, we can try to rate the condenser. It may be that the rating will fail under the operating conditions we set. In that case, we can try again with a higher number of tube rows, by altering some operating conditions (e.g., by lowering the temperature rise of the air), by choosing a different coil geometry, or by increasing the frontal area.

In [6]:
air_out, rfg_out = condenser.solve(
    air_in=air_in,
    air_m_dot=air_m_dot,
    rfg_in=rfg_in,
    rfg_m_dot=rfg_m_dot
)

Under the selected operating conditions and by setting the number of rows to 5, it follows that the condenser rating succeeds:

In [7]:
print(
    f"air out = {air_out.Tdb.to('degC'):~P.1f} DB, "
    f"{air_out.RH.to('pct'):~P.0f} RH\n"
    f"refrigerant out = {rfg_out.T.to('degC'):~P.1f}\n"
    f"degree of subcooling = {condenser.dT_sc.to('K'):~P.2f}\n"
    f"heat rejection rate = {condenser.Q_dot.to('kW'):~P.3f}\n"
    f"L2 desuperheating = {condenser.desuperheating_region.L_flow.to('mm'):~P.0f}\n"
    f"L2 condensing = {condenser.condensing_region.L_flow.to('mm'):~P.0f}\n"
    f"L2 subcooling = {condenser.subcooling_region.L_flow.to('mm'):~P.0f}\n"
    f"air-side pressure drop = {condenser.air_dP.to('Pa'):~P.3f}"
)

air out = 45.0 °C DB, 18 % RH
refrigerant out = 44.6 °C
degree of subcooling = 5.40 K
heat rejection rate = 9.203 kW
L2 desuperheating = 29 mm
L2 condensing = 73 mm
L2 subcooling = 8 mm
air-side pressure drop = 53.357 Pa


Note: the air-side pressure drop is also a factor to take into account. A large pressure drop would mean that we would need a large, energy-consuming fan to get the air flow through the condenser. 